# Piping a RunnableParallel with Other Runnables

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

c:\Users\Theo\source\repos\AUEB\LangChain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [3]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 500)

c:\Users\Theo\source\repos\AUEB\LangChain\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3641: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [4]:
string_parser = StrOutputParser()

In [5]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [6]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [7]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup and Requests.\n2. Developing a simple GUI-based application using Tkinter.\n3. Creating a basic Machine Learning model using Scikit-learn.'}

In [ ]:
# chain_time1 = (RunnableParallel({'books':chain_books, 
#                                 'projects':chain_projects}) 
#               | chat_template_time 
#               | chat 
#               | string_parser
#              )

In [8]:
chain_time2 = ({'books':chain_books, 
                'projects':chain_projects}
              | chat_template_time 
              | chat 
              | string_parser
             )

In [9]:
print(chain_time2.invoke({'programming language':'Python'}))

The time it takes to complete the literature and the projects can vary greatly depending on several factors such as your reading speed, comprehension level, the complexity of the projects, and the amount of time you can dedicate each day. 

However, here's a rough estimate:

1. "Fluent Python: Clear, Concise, and Effective Programming" - This book is around 800 pages. If you read and practice for about 2 hours a day, it might take you around 2-3 months to complete.

2. "Python Cookbook: Recipes for Mastering Python 3" - This book is around 700 pages. Again, if you read and practice for about 2 hours a day, it might take you around 2-3 months to complete.

3. "Effective Python: 90 Specific Ways to Write Better Python" - This book is around 230 pages. If you read and practice for about 2 hours a day, it might take you around 3-4 weeks to complete.

For the projects:

1. Building a Web Scraper using BeautifulSoup - If you're familiar with the basics of web scraping, this could take you an

In [10]:
chain_time2.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            